# Исследование надежности заемщиков


**Цель проекта:**  
На основе статистики о платёжеспособности клиентов исследовать влияет ли семейное положение, количество детей клиента, цель использования кредита на факт его возврата в срок.

**Для анализа используется датасет, содержащий следующие поля:**
* children — количество детей в семье
* days_employed — общий трудовой стаж в днях
* dob_years — возраст клиента в годах
* education — уровень образования клиента
* education_id — идентификатор уровня образования
* family_status — семейное положение
* family_status_id — идентификатор семейного положения
* gender — пол клиента
* income_type — тип занятости
* debt — имел ли задолженность по возврату кредитов
* total_income — ежемесячный доход
* purpose — цель получения кредита



## Обзор данных

In [ ]:
import pandas as pd

In [ ]:
try:
    data = pd.read_csv('/datasets/data.csv')
except:
    data = pd.read_csv('https://code.s3.yandex.net/datasets/data.csv')

Выведем первые 10 строчек датафрейма `data` на экран

In [ ]:
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Выведите основную информацию о датафрейме

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


**Выводы:**
- тип данных в столбцах соответствует их содержимому
- отсутствуют данные в столбцах days_employed, total_income  
  
Можно предположить следующие причины отсутствия данных:
- категорией занятости заемщика не предполагает внесения данных (напр.неработающий пенсионер)
- заемщик не предоставил данные
- при выгрузке данных произошла ошибка

## Предобработка данных

### Удаление пропусков

Выведем количество пропущенных значений для каждого столбца

In [ ]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Как было указано ранее, отсутствуют данные в столбцах days_employed, total_income.  
Столбец `days_employed` обработаем далее.  
Столбец с пропущенными значениями — `total_income` — хранит данные о доходах. На сумму дохода сильнее всего влияет тип занятости, поэтому заполним пропуски в этом столбце медианным значением по каждому типу из столбца `income_type`.

In [ ]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == t), 'total_income'].median()

### Обработка аномальных значений

В данных могут встречаться артефакты (аномалии) — значения, которые не отражают действительность и появились по какой-то ошибке.  
Таким артефактом будет отрицательное количество дней трудового стажа в столбце `days_employed`. Обработаем значения в этом столбце: заменим все отрицательные значения положительными.

In [ ]:
data['days_employed'] = data['days_employed'].abs()

Для каждого типа занятости выведем медианное значение трудового стажа `days_employed` в днях.

In [ ]:
data.groupby('income_type')['days_employed'].agg('median')

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64

У двух типов (безработные и пенсионеры) получатся аномально большие значения. Исправить такие значения сложно, поэтому оставим их как есть.

Выведем перечень уникальных значений столбца `children`.

In [ ]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

В столбце `children` есть два аномальных значения. Посмотрим, каков процент таких строк

In [ ]:
len(data.query('children == -1 or children == 20'))/len(data)*100

0.5714285714285714

Процент строк с некорректными значениями 0.6%. Удалим строки, в которых встречаются такие аномальные значения из датафрейма `data`.

In [ ]:
data = data[(data['children'] != -1) & (data['children'] != 20)]

Ещё раз выведем перечень уникальных значений столбца `children`, чтобы убедиться, что артефакты удалены.

In [ ]:
data['children'].unique()

array([1, 0, 3, 2, 4, 5])

### Удаление пропусков (продолжение)

Заполним пропуски в столбце `days_employed` медианными значениями для каждого типа занятости `income_type`.

In [ ]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[(data['income_type'] == t), 'days_employed'].median()

Убедимся, что все пропуски заполнены.

In [ ]:
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Изменение типов данных

Заменим вещественный тип данных в столбце `total_income` на целочисленный

In [ ]:
data['total_income'] = data['total_income'].astype(int)

### Обработка дубликатов

Обработаем неявные дубликаты в столбце `education`. В этом столбце есть одни и те же значения, но записанные по-разному: с использованием заглавных и строчных букв. Приведем их к нижнему регистру.

In [ ]:
data['education'] = data['education'].str.lower()

Выведем на экран количество строк-дубликатов в данных. Если такие строки присутствуют, удалим их.

In [ ]:
data.duplicated().sum()

71

In [ ]:
data = data.drop_duplicates()

### Категоризация данных

На основании диапазонов, указанных ниже, создадим в датафрейме `data` столбец `total_income_category` с категориями:

- 0–30000 — `'E'`;
- 30001–50000 — `'D'`;
- 50001–200000 — `'C'`;
- 200001–1000000 — `'B'`;
- 1000001 и выше — `'A'`.


Например, кредитополучателю с доходом 25000 назначим категорию `'E'`, а клиенту, получающему 235000, — `'B'`.  
Используем собственную функцию с именем `categorize_income()` и метод `apply()`.

In [ ]:
def categorize_income(income):
    try:
        if 0 <= income <= 30000:
            return 'E'
        elif 30001 <= income <= 50000:
            return 'D'
        elif 50001 <= income <= 200000:
            return 'C'
        elif 200001 <= income <= 1000000:
            return 'B'
        elif income >= 1000001:
            return 'A'
    except:
        pass

In [ ]:
data['total_income_category'] = data['total_income'].apply(categorize_income)

Выведем на экран перечень уникальных целей взятия кредита из столбца `purpose`

In [ ]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

Создадим функцию, которая на основании данных из столбца `purpose` сформирует новый столбец `purpose_category`, в который войдут следующие категории:

- `'операции с автомобилем'`,
- `'операции с недвижимостью'`,
- `'проведение свадьбы'`,
- `'получение образования'`.  
  
Используем собственную функцию с именем `categorize_purpose()` и метод `apply()`.  

In [ ]:
def categorize_purpose(row):
    try:
        if 'автом' in row:
            return 'операции с автомобилем'
        elif 'жил' in row or 'недвиж' in row:
            return 'операции с недвижимостью'
        elif 'свад' in row:
            return 'проведение свадьбы'
        elif 'образов' in row:
            return 'получение образования'
    except:
        return 'нет категории'

In [ ]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)

## Исследование данных

### Исследуем наличие зависимости между количеством детей и возвратом кредита в срок

Сгруппируем данные по количеству детей и сделаем сортировку по столбцу с долей невозвратов по возрастанию.  
Посчитаем доли просрочек для групп заемщиков с детьми и без.

In [ ]:
display(data.groupby('children')['debt'].agg({'mean','count'}).round(3).sort_values(by = 'mean'))
debt_with_child = data[data['children'] != 0]['debt'].mean().round(3)
debt_without_child = data[data['children'] == 0]['debt'].mean().round(3)
print(f'Заемщики с детьми: доля просрочки кредитов {debt_with_child:.1%}\n\
Заемщики без детей: доля просрочки кредитов {debt_without_child:.1%}')


,mean,count
children,,
5,0.000,9
0,0.075,14091
3,0.082,330
1,0.092,4808
2,0.095,2052
4,0.098,41


Заемщики с детьми: доля просрочки кредитов 9.2%
Заемщики без детей: доля просрочки кредитов 7.5%


**Вывод:**  
Учитывая релевантность исследуемой выборки, мы делаем вывод о том, что заемщики без детей чаще гасят свои обязательства в срок:
- просрочка платежа у заемщиков без детей 7.5%  
- просрочка платежа у заемщиков с детьми 9.2%

### Исследуем наличие зависимости между семейным положением и возвратом кредита в срок

Сгруппируем данные по семейному статусу и сделаем сортировку по столбцу с долей невозвратов по возрастанию

In [ ]:
data.groupby('family_status')['debt'].agg({'mean','count'}).round(2).sort_values(by = 'mean')

,mean,count
family_status,,
в разводе,0.07,1189
вдовец / вдова,0.07,951
женат / замужем,0.08,12261
гражданский брак,0.09,4134
Не женат / не замужем,0.10,2796


**Вывод:**  
Из таблицы мы видим, что выборка репрезентативна.  
Можно сделать вывод о том, что люди, которые имеют узаконенный семейный статус (вдовец / вдова, в разводе, женат / замужем) более своевременно гасят кредиты (просрочка  7-8%) в сравнении с людьми, статус которых не зафиксирован документально (гражданский брак, Не женат / не замужем) - просрочка  9-10%

### Исследуем наличие зависимости между уровнем дохода и возвратом кредита в срок

Сгруппируем данные по уровню дохода и сделаем сортировку по уровню дохода по убыванию

In [ ]:
data.groupby('total_income_category')['debt'].agg({'mean','count'}).round(3).\
sort_values(by = 'total_income_category')

,mean,count
total_income_category,,
A,0.080,25
B,0.071,5014
C,0.085,15921
D,0.060,349
E,0.091,22


**Вывод:**  
Для анализа возьмем 2 самые многочисленные выборки в исследуемых данных корреляции между уровнем дохода и погашением кредитов  в срок: люди с более низким доходом чаще допускают просрочки по кредитам.  

Просрочка по кредитам:
- у людей с доходом из кат."С" (50001 - 200000) - 8,5%
- у людей с доходом из кат."B" (200001 - 1000000) - 7,1%

### Исследуем закономерность, как разные цели кредита влияют на его возврат в срок?

Сгруппируем данные по цели кредитования и сделаем сортировку по столбцу с долей невозвратов по возрастанию

In [ ]:
data.groupby('purpose_category')['debt'].agg({'mean','count'}).round(2).sort_values(by = 'mean')

,mean,count
purpose_category,,
операции с недвижимостью,0.07,10751
проведение свадьбы,0.08,2313
операции с автомобилем,0.09,4279
получение образования,0.09,3988


**Вывод:**  
Данные говорят о том, что меньше всего просрочек по кредитам по направлению "операции с недвижимостью" - 7%  
На 2-м месте по возврату в срок кредиты на "проведение свадьбы"- 8%  
Больше всего просрочек по кредитам по направлениям "операции с автомобилем" и "получение образования" - 9%

### Проверим предположение о том, что существует взаимосвязь одновременно с семейным статусом, наличием детей и возвратом кредита в срок

In [ ]:
print('Вдовцы с 2 детьми выборке:', len(data[(data['family_status'] == 'вдовец / вдова') & (data['children'] == 2)]))
data.pivot_table(index = 'family_status', columns = 'children', values = 'debt', aggfunc = 'mean').round(2)

Вдовцы с 2 детьми выборке: 20


children,0,1,2,3,4,5
family_status,,,,,,
Не женат / не замужем,0.09,0.12,0.12,0.12,0.5,NaN
в разводе,0.07,0.07,0.09,0.09,0.0,NaN
вдовец / вдова,0.06,0.09,0.15,0.00,0.0,NaN
гражданский брак,0.08,0.12,0.09,0.14,0.0,0.0
женат / замужем,0.07,0.08,0.09,0.07,0.1,0.0


**Вывод:**  
Данные говорят о том, что люди с семейным статусом "Не женат / не замужем" имеющие детей - самые недобросовестные заемщики по сравнению с другими категориями в анализируемой выборке

## Вывод по результатам исследования

Проведено исследование для кредитного отдела банка по оценке платежеспособности клиентов в зависимости  от анализируемых критериев.
    
<p></p>

На входе данные от банка, которые содержат выборку клиентов в количестве 21525 чел., включающую 11 критериев:
* children — количество детей в семье
* days_employed — общий трудовой стаж в днях
* dob_years — возраст клиента в годах
* education — уровень образования клиента
* education_id — идентификатор уровня образования
* family_status — семейное положение
* family_status_id — идентификатор семейного положения
* gender — пол клиента
* income_type — тип занятости
* debt — имел ли задолженность по возврату кредитов
* total_income — ежемесячный доход
* purpose — цель получения кредита
    
<p></p>

**На этапе предобработки были обнаружены следующие недостатки в данных, которые устранены:**
- отрицательные значения в столбце days_employed заменены на положительные
- пропуски в столбцах days_employed, total_income заменены медианными значениями
- данные столбца total_income приведены к типу int
- данные по доходам total_income разбиты на категории в новом столбце total_income_category
- данные по целям получения кредита purpose разбиты на категории в новом столбце purpose_category
</div>

<div style="border:solid steelblue 2px; padding: 10px; border-radius: 5px">
<p></p>

**На основании проведенного исследования сделали следующие выводы:**
- существует зависимость между наличием детей и возвратом кредита в срок:
    - процент просрочек у людей с детьми 9,2%
    - процент просрочек у людей без детей 7,5%
    
<p></p>

- существует зависимость между семейным положением и возвратом кредита в срок:
    - люди, состоящие\состоявшие в браке имеют процент просрочек 7-8%
    - люди никогда не состоявшие в официальном браке имеют просрочку 9-10%
        
<p></p>


- существует зависимость между уровнем дохода и возвратом кредита в срок:
    - у людей с доходом до 200000 руб. просрочка выше - 8,5%
    - у людей с доходом свыше 200001 руб. просрочка сотавляет 7,1%
    
<p></p>

- разные цели кредита влияют на его возврат в срок:
    - ниже остальных просрочка по кредитам на недвижимость - 7%
    - просрочка по кредитам на свадьбу - 8%
    - самые проблемные кредиты на "операции с автомобилем" и "получение образования" - 9%
        
<p></p>

- в результате более детального анализа сделали вывод: больше всего просрочек по кредитам у людей с семейным статусом "не женат/не замужем" имеющих детей - 12%
        
<p></p>    
    
**Рекомендации по улучшению сбора данных:**
    
1. Рекомендуется сделать ограниченный список для выбора цели кредита

2. В случае отсутствия данных добавить признак отсутствия причины:
  - не предоставлены клиентом
  - отсутствуют
  - прочее